In [7]:
import pandas as pd
import json

## Check los valores del json

In [4]:
json_path:str = "./data/school_data.json"

In [8]:
with open(json_path,encoding="utf8") as f:
    data = f.read()
    data = json.loads(data)

### Podemos observar que esto permite realizar una unnest data

In [10]:
pd.json_normalize(data,record_path="students")

,id,name,math,physics,chemistry
0,A001,Tom,60,66,61
1,A002,James,89,76,51
2,A003,Jenny,79,90,78


### Data nesteada y agregada a cada uno de los datos nesteados

In [12]:
pd.json_normalize(
    data, 
    record_path =['students'], 
    meta=['school_name', 'class']
)


,id,name,math,physics,chemistry,school_name,class
0,A001,Tom,60,66,61,ABC primary school,Year 1
1,A002,James,89,76,51,ABC primary school,Year 1
2,A003,Jenny,79,90,78,ABC primary school,Year 1


# SQLITE

In [13]:
import sqlite3 as sql
import pandas as pd
import matplotlib.pyplot as plt

In [21]:

sqlite_data_path = "./data/nba_salary.sqlite" 
connection = sql.connect(sqlite_data_path)
cursor = connection.cursor()

In [23]:
query_string = """
    SELECT 
        name
    FROM sqlite_master
    WHERE
        type = 'table' 
"""

cursor.execute(query_string)

for row in cursor.fetchall():
    print(row)


('NBA_season1718_salary',)
('Seasons_Stats',)


In [24]:
query_nba = """
SELECT 
    * 
FROM NBA_season1718_salary
"""
query_season = """
SELECT 
    * 
FROM Seasons_Stats
"""

In [25]:

df_salary = pd.read_sql_query(query_nba, connection)
df_salary.head()

,X1,Player,Tm,season17_18
0,1.0,Stephen Curry,GSW,34682550.0
1,2.0,LeBron James,CLE,33285709.0
2,3.0,Paul Millsap,DEN,31269231.0
3,4.0,Gordon Hayward,BOS,29727900.0
4,5.0,Blake Griffin,DET,29512900.0


In [27]:

df_stats = pd.read_sql_query(query_season, connection)
df_stats.head()

,X1,Year,Player,Pos,Age,Tm,G,GS,MP,PER,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,0.0,1950.0,Curly Armstrong,G-F,31.0,FTW,63.0,NaN,NaN,NaN,...,0.705,NaN,NaN,NaN,176.0,NaN,NaN,NaN,217.0,458.0
1,1.0,1950.0,Cliff Barker,SG,29.0,INO,49.0,NaN,NaN,NaN,...,0.708,NaN,NaN,NaN,109.0,NaN,NaN,NaN,99.0,279.0
2,2.0,1950.0,Leo Barnhorst,SF,25.0,CHS,67.0,NaN,NaN,NaN,...,0.698,NaN,NaN,NaN,140.0,NaN,NaN,NaN,192.0,438.0
3,3.0,1950.0,Ed Bartels,F,24.0,TOT,15.0,NaN,NaN,NaN,...,0.559,NaN,NaN,NaN,20.0,NaN,NaN,NaN,29.0,63.0
4,4.0,1950.0,Ed Bartels,F,24.0,DNN,13.0,NaN,NaN,NaN,...,0.548,NaN,NaN,NaN,20.0,NaN,NaN,NaN,27.0,59.0


## Join data from dataframes

In [30]:
merged_data = df_salary.merge(
        right=df_stats
    ,   on = "Player"
)

In [51]:
merged_data.fillna(0,inplace=True)

In [52]:
merged_data.query("Player == 'Aaron Gray'")

,X1_x,Player,Tm_x,season17_18,X1_y,Year,Pos,Age,Tm_y,G,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
3032,492.0,Aaron Gray,DET,452059.0,18949.0,2008.0,C,23.0,CHI,61.0,...,0.566,0.0,0.0,168.0,41.0,0.0,0.0,0.0,117.0,262.0
3033,492.0,Aaron Gray,DET,452059.0,19550.0,2009.0,C,24.0,CHI,56.0,...,0.576,0.0,0.0,220.0,46.0,0.0,0.0,0.0,135.0,196.0
3034,492.0,Aaron Gray,DET,452059.0,20120.0,2010.0,C,25.0,TOT,32.0,...,0.714,0.0,0.0,106.0,22.0,0.0,0.0,0.0,54.0,104.0
3035,492.0,Aaron Gray,DET,452059.0,20121.0,2010.0,C,25.0,CHI,8.0,...,0.286,0.0,0.0,16.0,2.0,0.0,0.0,1.0,10.0,18.0
3036,492.0,Aaron Gray,DET,452059.0,20122.0,2010.0,C,25.0,NOH,24.0,...,0.857,0.0,0.0,90.0,20.0,0.0,0.0,0.0,44.0,86.0
3037,492.0,Aaron Gray,DET,452059.0,20751.0,2011.0,C,26.0,NOH,41.0,...,0.500,0.0,0.0,171.0,16.0,0.0,0.0,0.0,96.0,129.0
3038,492.0,Aaron Gray,DET,452059.0,21327.0,2012.0,C,27.0,TOR,49.0,...,0.532,0.0,0.0,281.0,27.0,0.0,0.0,0.0,127.0,191.0
3039,492.0,Aaron Gray,DET,452059.0,21872.0,2013.0,C,28.0,TOR,42.0,...,0.523,0.0,0.0,133.0,33.0,0.0,0.0,0.0,85.0,119.0
3040,492.0,Aaron Gray,DET,452059.0,22486.0,2014.0,C,29.0,TOT,37.0,...,0.550,0.0,0.0,111.0,22.0,0.0,0.0,0.0,64.0,65.0
3041,492.0,Aaron Gray,DET,452059.0,22487.0,2014.0,C,29.0,TOR,4.0,...,0.500,0.0,0.0,8.0,3.0,0.0,0.0,0.0,7.0,5.0


In [55]:
cols = merged_data.select_dtypes(include='number').columns
cols

Index(['X1_x', 'season17_18', 'X1_y', 'Year', 'Age', 'G', 'GS', 'MP', 'PER',
       'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%',
       'TOV%', 'USG%', 'blanl', 'OWS', 'DWS', 'WS', 'WS/48', 'blank2', 'OBPM',
       'DBPM', 'BPM', 'VORP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P',
       '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST',
       'STL', 'BLK', 'TOV', 'PF', 'PTS'],
      dtype='object')

In [56]:
median_data = merged_data.groupby("Player")[cols].median()

In [57]:
median_data

,X1_x,season17_18,X1_y,Year,Age,G,GS,MP,PER,TS%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
Player,,,,,,,,,,,,,,,,,,,,,
A.J. Hammons,411.0,1312611.0,24304.0,2017.0,24.0,22.0,0.0,163.0,8.40,0.4720,...,0.450,0.0,0.0,36.0,4.0,1.0,0.0,0.0,21.0,48.0
Aaron Brooks,319.0,2116955.0,21752.0,2013.0,28.0,53.0,0.0,959.0,12.50,0.5210,...,0.833,0.0,0.0,78.0,129.0,0.0,0.0,0.0,93.0,376.0
Aaron Gordon,190.0,5504420.0,23698.0,2016.0,20.0,78.0,0.0,1863.0,14.40,0.5300,...,0.719,0.0,0.0,405.0,128.0,0.0,0.0,0.0,153.0,719.0
Aaron Gray,492.0,452059.0,20751.0,2011.0,26.0,37.0,0.0,355.0,11.00,0.5290,...,0.550,0.0,0.0,111.0,22.0,0.0,0.0,0.0,64.0,104.0
Al Horford,11.0,27734405.0,21646.0,2012.5,25.5,75.0,0.0,2429.0,19.40,0.5650,...,0.746,0.0,0.0,610.0,214.5,0.0,0.0,0.0,163.0,1050.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wilson Chandler,101.0,12016854.0,20630.0,2011.0,23.0,62.0,0.0,1927.0,13.20,0.5260,...,0.795,0.0,0.0,303.0,114.0,0.0,0.0,0.0,172.0,846.0
Yogi Ferrell,429.0,1312611.0,24260.0,2017.0,23.0,36.0,0.0,1046.0,13.10,0.5330,...,0.831,0.0,0.0,99.0,155.0,0.0,0.0,0.0,79.0,408.0
Zach LaVine,251.0,3202217.0,23828.0,2016.0,20.0,77.0,0.0,1902.0,14.30,0.5480,...,0.836,0.0,0.0,214.0,251.0,0.0,0.0,0.0,158.0,889.0


# GITHUB API

In [1]:
import requests

In [2]:
user:str = "Andru-1987"
repos:str = "repos"
endpoint:str = f"https://api.github.com/users/{user}/{repos}"

### GET method to `github` data

In [33]:
response_json = requests.get(endpoint).json()

In [34]:
data_by_list_api = pd.DataFrame(response_json)

In [35]:
data = data_by_list_api[["id","name","full_name","private","owner","html_url","size"]]

In [36]:
# no me sirve nada
pd.json_normalize(data_by_list_api[data_by_list_api.id == 773098613].owner)

,login,id,node_id,avatar_url,gravatar_id,url,html_url,followers_url,following_url,gists_url,starred_url,subscriptions_url,organizations_url,repos_url,events_url,received_events_url,type,site_admin
0,Andru-1987,86977326,MDQ6VXNlcjg2OTc3MzI2,https://avatars.githubusercontent.com/u/869773...,,https://api.github.com/users/Andru-1987,https://github.com/Andru-1987,https://api.github.com/users/Andru-1987/followers,https://api.github.com/users/Andru-1987/follow...,https://api.github.com/users/Andru-1987/gists{...,https://api.github.com/users/Andru-1987/starre...,https://api.github.com/users/Andru-1987/subscr...,https://api.github.com/users/Andru-1987/orgs,https://api.github.com/users/Andru-1987/repos,https://api.github.com/users/Andru-1987/events...,https://api.github.com/users/Andru-1987/receiv...,User,False


In [39]:
data.drop("owner", axis=1, inplace=True)

/tmp/ipykernel_1212/1345291773.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.drop("owner", axis=1, inplace=True)


In [40]:
data.head()

,id,name,full_name,private,html_url,size
0,771739188,24150-cac-fullstack-python,Andru-1987/24150-cac-fullstack-python,False,https://github.com/Andru-1987/24150-cac-fullst...,2007
1,750482076,53175-sql,Andru-1987/53175-sql,False,https://github.com/Andru-1987/53175-sql,958
2,762708815,53180-sql,Andru-1987/53180-sql,False,https://github.com/Andru-1987/53180-sql,919
3,803780561,57190-sql-coderhouse,Andru-1987/57190-sql-coderhouse,False,https://github.com/Andru-1987/57190-sql-coderh...,121
4,787897212,61875-data-engineering,Andru-1987/61875-data-engineering,False,https://github.com/Andru-1987/61875-data-engin...,162
